# 장바구니 상관관계 분석

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

csv 파일 불러올 때 encoding

한글 encoding : euc-kr, cp949
엑셀로 불러온다음 인코딩을 강제로 해준다.

컬럼때문에 안불러와지면 컬럼 삭제하고 나중에 가져오면된다.

예외처리를해서 euc-kr, cp949 둘다 해본다.

try
  euc-kr
except:
  cp949

1번 데이터와 2번 데이터를 비교한다면

1번과 2번의 기준이 되는 index가 똑같아야한다.

1번과 2번의 기준이되는 index가 똑같아야한다.

In [2]:
#데이터 셋의 형태는 이런 구조여야 장바구니 분석이 가능하다.
dataset = [
    ['커피','녹차','얼음'], #1인 구매 리스트
    ['커피','얼음','우유'],
    ['커피','딸기'],
    ['바나나','녹차','딸기'],
    ['커피','얼음'],
    ['우유','커피'],
    ['우유','딸기'],
    ['커피'],
    ['얼음','커피']
]

import pandas as pd
#장바구니 분석을 하기위해 필요한 패키지들
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [3]:
#학습
te = TransactionEncoder()
#데이터셋을 fit으로 맞추고 transform으로 바꿔준다.
te_ary = te.fit(dataset).transform(dataset)
te_ary

array([[ True, False, False,  True, False,  True],
       [False, False, False,  True,  True,  True],
       [False,  True, False, False, False,  True],
       [ True,  True,  True, False, False, False],
       [False, False, False,  True, False,  True],
       [False, False, False, False,  True,  True],
       [False,  True, False, False,  True, False],
       [False, False, False, False, False,  True],
       [False, False, False,  True, False,  True]])

In [4]:
#그냥써도 되지만 데이터프레임으로 넣어주면 보기좋아진다.
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,녹차,딸기,바나나,얼음,우유,커피
0,True,False,False,True,False,True
1,False,False,False,True,True,True
2,False,True,False,False,False,True
3,True,True,True,False,False,False
4,False,False,False,True,False,True
5,False,False,False,False,True,True
6,False,True,False,False,True,False
7,False,False,False,False,False,True
8,False,False,False,True,False,True


In [5]:
#apriori 알고리즘 사용
f_data = apriori(df,use_colnames=True) #min_support옵션을 주지않으면 기본 지지도는 0.5로 설정된다.
f_data

,support,itemsets
0,0.777778,(커피)


In [6]:
#이번에는 옵션을 넣어 지지도를 낮춰서 본다.
f_data = apriori(df,min_support=0.1, use_colnames=True)
f_data

,support,itemsets
0,0.222222,(녹차)
1,0.333333,(딸기)
2,0.111111,(바나나)
3,0.444444,(얼음)
4,0.333333,(우유)
5,0.777778,(커피)
6,0.111111,"(녹차, 딸기)"
7,0.111111,"(녹차, 바나나)"
8,0.111111,"(녹차, 얼음)"
9,0.111111,"(녹차, 커피)"


In [7]:
from mlxtend.frequent_patterns import association_rules
#지지도(support) 전체거래항목중 a,b를 포함하는 비율
#신뢰도(confidence) a를 포함하는 거래중에 a,b가 동시에 거래되는 비중
#향상도(Lift) a와 b사이에 상호관계가 없으면 1 //// 1보다 큰값 연관성이 낮다

#threshold = 신뢰도
#
#min_threshold의 기본은 0.8이다.
association_rules(f_data, metric='confidence', min_threshold= 0.6)

# association_rules(f_data, metric='lift', min_threshold= 0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(바나나),(녹차),0.111111,0.222222,0.111111,1.000000,4.500000,0.086420,inf
1,(바나나),(딸기),0.111111,0.333333,0.111111,1.000000,3.000000,0.074074,inf
2,(얼음),(커피),0.444444,0.777778,0.444444,1.000000,1.285714,0.098765,inf
3,(우유),(커피),0.333333,0.777778,0.222222,0.666667,0.857143,-0.037037,0.666667
4,"(녹차, 딸기)",(바나나),0.111111,0.111111,0.111111,1.000000,9.000000,0.098765,inf
5,"(녹차, 바나나)",(딸기),0.111111,0.333333,0.111111,1.000000,3.000000,0.074074,inf
6,"(바나나, 딸기)",(녹차),0.111111,0.222222,0.111111,1.000000,4.500000,0.086420,inf
7,(바나나),"(녹차, 딸기)",0.111111,0.111111,0.111111,1.000000,9.000000,0.098765,inf
8,"(녹차, 커피)",(얼음),0.111111,0.444444,0.111111,1.000000,2.250000,0.061728,inf
9,"(녹차, 얼음)",(커피),0.111111,0.777778,0.111111,1.000000,1.285714,0.024691,inf


# 해석

lift(향상도) 수치가 1보다 큰 것들이 있는데, 1보다 클수록 상관관계가 있다.
1이거나, 1보다 작으면 아무관계 없다.

위의 표를 해석해보면 바나나 구매하면 녹차, 딸기를 구매하는 경우가 있다. 둘의 신뢰도가 0.111111로 같다

이중 녹차와 딸기 둘다 향상도를 보면 1보다 크기때문에 높은 인기를 가진다는 것을 알수있다.

In [8]:
# import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [11]:
#방법 1
import csv
 
f = open('data2/groceries.csv', 'r')
rdr = csv.reader(f)
k = []
for l in rdr:
    k.append(l)

In [14]:
#방법 2
f=open('data2/groceries.csv','r',encoding='utf-8')
df2 = []
lines = f.readlines()
dataset = []
for i in lines:
    dataset.append(i.split(','))
dataset

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups\n'],
 ['tropical fruit', 'yogurt', 'coffee\n'],
 ['whole milk\n'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads\n'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product\n'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner\n'],
 ['rolls/buns\n'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)\n'],
 ['potted plants\n'],
 ['whole milk', 'cereals\n'],
 ['tropical fruit',
  'other vegetables',
  'white bread',
  'bottled water',
  'chocolate\n'],
 ['citrus fruit',
  'tropical fruit',
  'whole milk',
  'butter',
  'curd',
  'yogurt',
  'flour',
  'bottled water',
  'dishes\n'],
 ['beef\n'],
 ['frankfurter', 'rolls/buns', 'soda\n'],
 ['chicken', 'tropical fruit\n'],
 ['butter', 'sugar', 'fruit/vegetable juice', 'newspapers\n'],
 ['fruit/vegetable juice\n'],
 ['packaged fruit/vegetables\n'],
 ['chocolate\n'],
 ['specialty bar\

In [16]:
#방법 3
df = pd.read_csv('data2/groceries.csv',sep='\t',header=None)
df

,0
0,"citrus fruit,semi-finished bread,margarine,rea..."
1,"tropical fruit,yogurt,coffee"
2,whole milk
3,"pip fruit,yogurt,cream cheese,meat spreads"
4,"other vegetables,whole milk,condensed milk,lon..."
...,...
9830,"sausage,chicken,beef,hamburger meat,citrus fru..."
9831,cooking chocolate
9832,"chicken,citrus fruit,other vegetables,butter,y..."
9833,"semi-finished bread,bottled water,soda,bottled..."


In [17]:
dataset = []
for temp in df[0]:
    dataset.append(temp.split(','))

In [18]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary,columns=te.columns_)
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,False
9831,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9832,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9833,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.110524,(bottled water)
1,0.193493,(other vegetables)
2,0.183935,(rolls/buns)
3,0.108998,(root vegetables)
4,0.174377,(soda)
5,0.104931,(tropical fruit)
6,0.255516,(whole milk)
7,0.139502,(yogurt)


In [20]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric='lift', min_threshold= 1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


# 웹툰크롤링

In [21]:
#https://comic.naver.com/webtoon/genre
#제목가져와
# 셀레니움 html의 값들을 검색해줄수있는게 beautifulsoup

In [22]:
import requests
from bs4 import BeautifulSoup

In [23]:
div or ul 최상단까지 올라가는게 처음

처음에 맨위에 있는 태그를 검색해서 데이터 들어있는지 안들어있는지 확인 그리고 밑에가서 확인 마지막에 최종데이터 가져오기

SyntaxError: invalid syntax (<ipython-input-23-52bef2dcb3aa>, line 1)

In [24]:
#페이지를 적어주고
#이 주소의 html 데이터를 한꺼번에 불러온다.
page = requests.get('https://comic.naver.com/webtoon/genre.nhn')
#가져와서 화면에 출력을 해봄,,, 원하는 데이터가 있는지 확인
#여기에서 안보이면 과감하게 크롤링 포기하기, 없으면 막혀있는거
html = page.text
html

'\r\n\r\n\r\n  \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t\r\n\t\t\r\n\t\t\r\n\t\t\r\n\t\t\r\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n\t\t\r\n\t<meta http-equiv="Content-type" content="text/html; charset=UTF-8">\r\n\t<title>네이버 만화 &gt; 웹툰 &gt; 장르별 웹툰</title>\r\n\t\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n        <meta property="og:title" content="네이버 웹툰" >\r\n        <meta property="og:image" content="https://ssl.pstatic.net/static/comic/images/og_tag_v2.png" >\r\n        <meta property="og:description" content="매일매일 새로운 재미, 네이버 웹툰.">\r\n    \r\n\r\n\r\n<meta property="og:url" content="https://comic.naver.com/webtoon/genre" >\r\n<meta property="og:type" content="article" >\r\n<meta property="og:article:author" content="네이버 웹툰" >\r\n<meta property="og:article:author:url" content="https://comic.nave

In [ ]:
# BeautifulSoul에는 가져온걸 나눠주는애가 두가지 있는데 html과 lxml 이 있다. 그 중에 가장 빠른게 lxml이다.
#첫번째가 가져온 데이터, 두번째는 html 태그 값을 찾을 떄 쓰는 파싱할때 쓰는 패키지를 선언해준다.
#html.parser => 안깔아도 된다. 하지만 속도가 느리다.
#lxml(pip install 로 따로 설치해서 써야함) => 속도가 빠르다.
#html5lib => html5형식으로 할수있따...? 잘안쓰나봄



# soup.find() => 태그 한개를 찾을 때
# soup.find_all => 모든 태그를 찾아서 가져옴  ///  저장형태 : 리스트 ['<div ...></div>']
# soup.find('div', class_='클래스명') //// soup.find('div',{'class or id':'list_area'}) 방법이 여러가지
soup = BeautifulSoup(html,'lxml')

In [ ]:
# soup.find = 조건에 맞는 태그 하나만 가져옴
# soup.find_all = 조건에 맞는 모든 태그를 가져옴
# 클래스명으로 찾기

result = bs4.find('div',class_='ex_class') print(result)

#id명으로 찾기

result = bs4.find('div', id='ex_id') print(result)

#참고사이트 https://brownbears.tistory.com/414

In [ ]:
# 딕셔너리 형식으로도 찾는게 가능하다
data1 = soup.find('div', {'class' : 'list_area'})
data1

In [ ]:
#앞에서 저장한 데이터 에서 찾는다
#저장 방법은 csv파일로 저장하는게 가장좋다.
import csv

data2 = data1.find_all('dt')
result = []
for temp in data2:
    result.append(temp.find('a').attrs['title'])
    
print(result)

In [ ]:
#csv로 만들기 1
import pandas as pd

df = pd.DataFrame(result)
#한글 저장이기 때문에 인코딩해주고 인덱스 없애야한다.
#저장할때 옵션값을 줄 수 있다.
#mode = 'w' : 새로만들 때, 'a' : 뒤에 이어붙임
# header = False 하면 헤더값도 안들어간다.
df.to_csv('이어서 저장.csv', index = False, encoding='euc-kr', mode='a', header = False)

In [ ]:
df1 = pd.read_csv('제목.csv',encoding='euc-kr')
df1

In [ ]:
#2번째 방법
#열었으면 닫아줘야한다.
f = open('웹툰제목크롤링 데이터.csv', 'w')
for temp in result:
    print(temp)
    f.write(temp + ',')# \n 쓰면 한줄마다 띄어쓰기
f.close

In [ ]:
ydata = [
    ['타이틀',2,2],
    ['타이틀2', 100, 300],
    ['타이틀3', 200, 400],
]
f = open('유투브데이터.csv','w')
f.write('채널명' + ',' + '조회수' + ',' + '좋아요' + '\n')
for temp in ydata:
    f.write(temp[0].replace(',','') + ',' + str(temp[1]) + ',' + str(temp[2]) + '\n')
f.close()

#구분자를 컴마로 하면 문제가 많이 생긴다. replace를 해준다.

In [ ]:
#한글, 영어, 숫자만 남기고 특수문자를 제거하는 방법
import re

string = 'abcd가나다라마바사12345[]!#!#!$!$$'

#정규표현식
re.sub('[^A-Za-z0-9가-힣]','',string)

In [ ]:
ydata = [
    ['타이틀',2,2],
    ['타이틀2', 100, 300],
    ['타이틀3', 200, 400],
]
col = ['제목','좋아요','조회수']
dic={}
i=0
j=0
result = []
for temp1 in ydata:
    for temp2 in temp1:
        dic[col[i]] = temp2[i]
    result.append(dic)

# 네이버 뉴스 헤더 긁어오기


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
#네이버 정책상 user-agent를 넣어줘야 크롤링이가능하다
headers = {'User-Agent' : 'mozilla/5.0', 'Accept-Encoding':'gzip,deflate','Accept':'*/*','Connection':'keep-alive'}
#url과 user-agent 넣어준다.
page = requests.get('https://news.naver.com', headers = headers)
#page의 모든 text를 받아온다.
html = page.text
html

In [ ]:
#beautifulsoup를 생성 후 담아준다.(슬라이싱)
soup = BeautifulSoup(html,'lxml')

In [ ]:
temp = soup.find('ul',{'class':'hdline_article_list'})

temp1 = temp.find_all('div',{'class':'hdline_article_tit'})

titlelist = []
for temp2 in temp1:
    #strip()를 해주면 텍스트 제외하고 공백 및 특수문자 이런게 다 사라짐
    titlelist.append(temp2.text.strip())

titlelist

In [ ]:
#csv로 만들기

# 뉴스 내용까지 긁어오기

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = {'User-Agent' : 'mozilla/5.0', 'Accept-Encoding':'gzip,deflate','Accept':'*/*','Connection':'keep-alive'}
page = requests.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%BB%A4%ED%94%BC', headers = headers)
html = page.text
html

In [ ]:
soup = BeautifulSoup(html,'lxml')

In [ ]:
data = soup.find('ul',{'class':'list_news'})
data

In [ ]:
data1 = data.find_all('a',{'class':'news_tit'})
title = []
for data2 in data1:
    title.append(data2.text)
title

In [ ]:
data3 = data.find_all('div',{'class':'dsc_wrap'})

In [ ]:
data3
body = []
for data3 in data3:
    body.append(data3.text)
body

In [ ]:
import pandas as pd
col = ['제목','내용']
a = pd.DataFrame([title])
a = a.transpose()
a

In [ ]:
b = pd.DataFrame([body])
b = b.transpose()
b

In [ ]:
all = pd.concat([a,b],axis=1)
all.columns = ['제목','내용']
all

# 한페이지만 검색

In [ ]:
#파일을 생성하기 때문에 한번에 다 써야한다.
from random import
time.sleep(random.randrange(1,3)) #크롤링 많이 하면 네이버가 막아서 타임 설정

import requests
from bs4 import BeautifulSoup
import urllib
import re

keyword = input('검색하실 키워드를 입력하세요(ex:커피,코로나...) : ')
keyword1 = urllib.parse.quote(keyword)

#agent 넣어줘야함
headers = {'User-Agent' : 'mozilla/5.0', 'Accept-Encoding':'gzip,deflate','Accept':'*/*','Connection':'keep-alive'}
page = requests.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query={0}'.format(keyword1))
html = page.text

soup = BeautifulSoup(html, 'lxml')

data = soup.find('div',{'id':'main_pack'})

data2 = data.find_all('div', {'class':'news_area'})

#제목
result_title = []

for temp in data2:
    titletemp = temp.find('a',{'class':'news_tit'})
    result_title.append(re.sub('[^A-Za-z0-9가-힣]',' ',titletemp.attrs['title']))


data3 = data.find_all('a', {'class':'api_txt_lines dsc_txt_wrap'})
#내용
result_desc = []

for temp in data3:
    result_desc.append(re.sub('[^A-Za-z0-9가-힣]', ' ', temp.text))

    #정규표현식
    #^의 의미는 문자열의 맨 처음과 일치한다

result = []

for temp in range(0, 10):
    temp1 = []
    temp1.append(result_title[temp])
    temp1.append(result_desc[temp])
    
    result.append(temp1)


f = open('{0} 데이터.csv'.format(keyword), 'w')
f.write('제목' +',' + '내용' + '\n')

for temp in result:
    f.write(temp[0]+','+temp[1]+'\n')
f.close()



# 10페이지까지 검색

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import re

keyword = input('검색하실 키워드를 입력하세요(ex:커피,코로나...) : ')
keyword1 = urllib.parse.quote(keyword)
for i in range(1,101,10):
   
       url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={0}&start={1}'.format(keyword,temp)
    
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html,'lxml')
    data1 = soup.find('div',{'id':'main_pack'})
    data2 = data1.find_all('div',{'class':'news_area'})
    import re

    result_title = []

    for temp in data2:
        titletemp = temp.find('a',{'class':'news_tit'})
        result_title.append(re.sub('[^A-Za-z0-9가-힣]',' ',titletemp.attrs['title']))


    data3 = data.find_all('a', {'class':'api_txt_lines dsc_txt_wrap'})

    result_desc = []

    for temp in data3:
        result_desc.append(re.sub('[^A-Za-z0-9가-힣]', ' ', temp.text))

    result = []

    for temp in range(0, 10):
        temp1 = []
        temp1.append(result_title[temp])
        temp1.append(result_desc[temp])

        result.append(temp1)


    f = open('{0} 데이터.csv'.format(keyword), 'a')
    if i == 1:
        f.write('제목' +',' + '내용' + '\n')
    
    for temp in result:
        f.write(temp[0]+','+temp[1]+'\n')
    f.close()



# 사진 크롤링

In [ ]:
import urllib.request
import requests
import time
from bs4 import BeautifulSoup


url = 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMTA4MzFfNjkg%2FMDAxNjMwMzkwNzYzMTI3.AAdvwiu2rs-wftQWbG6_BhKgW9-jXwBNQBp5imv54awg.MDhB5FwTRGkwNEg2ZJwK2JloQ73XrW7C2P0zwvLqAH4g.PNG.positive-yang%2Fimage.png&type=a340'

urllib.request.urlretrieve(url, '커피1.jpg') #폴더생성


# 네이버 쇼핑검색 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import urllib
import time

In [ ]:
#json 데이터 구조
result = ' {"name":"홍길동" , "age":29} '

#json 데이터 불러오기
jdata = json.loads(result)

In [ ]:
#검색하고 싶은 키워드를 받는다.
keyword = input('네이버 쇼핑 검색어 : ')
#한글로 바로넣으면 오류가나기 때문에 변환
keyword = urllib.parse.quote(keyword)

#url을 넣어 데이터를 받아온다.
page = requests.get('https://search.shopping.naver.com/search/all?where=all&frm=NVSCTAB&query={0}'.format(keyword))

#받아온 페이지를 텍스트만 가져옴
html = page.text
html

In [ ]:
#html태그들을 나눠준다.
soup = BeautifulSoup(html,'lxml')


#json데이터를 찾는다. 
# tip : 검색기능을 이용하여 { 이게 시작하는곳을 찾으면 빠름
jsonresult = soup.find('script',{'id':'__NEXT_DATA__'})


In [ ]:
#json 데이터를 로드한다.(str형식으로)
jsondata = json.loads(jsonresult.string)

#우리가 원하는 내용이 있는곳까지 들어가야해서 하나하나씩 json데이터를 확인
jdata1 = jsondata['props']['pageProps']['initialState']['products']['list']

#결과를 담을 리스트 생성
result = []


#json데이터를 for문에 넣고 하나씩 꺼내온다.
for temp in jdata1:
    #꺼내온 데이터를 저장할 리스트를 생성
    rtemp = []
    #item이름이 어디있는지 찾아야함 ;;;
    rtemp.append(temp['item']['productTitle'])
    #가격을 찾아야함
    rtemp.append(temp['item']['lowPrice'])
    #result에 가져온 값을 넣는다.
    result.append(rtemp)
    
result
    

# 쇼핑검색2

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import urllib
import time

In [ ]:
# 아까랑 똑같음
keyword = input('네이버 쇼핑 검색어 : ')
keyword = urllib.parse.quote(keyword)

result = []


#이건 어제 했던건데 페이지 넘길때마다 주소 어디부분이 바뀌는지 체크후 만들어주면된다.
#체크후 크롬에 주소 넣어봐서 맞는지 확인
for pagenum in range(1,11):
    url = 'https://search.shopping.naver.com/search/all?frm=NVSCTAB&origQuery={0}&pagingIndex={1}&pagingSize=40&productSet=total&query={0}&sort=rel&timestamp=&viewType=list'.format(keyword,pagenum)
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html,'lxml')
    jsonresult = soup.find('script',{'id':'__NEXT_DATA__'})
    
    jsondata = json.loads(jsonresult.string)
    
    
    
    jdata1 = jsondata['props']['pageProps']['initialState']['products']['list']

    for temp in jdata1:
        rtemp = []
        rtemp.append(temp['item']['productTitle'])
        rtemp.append(temp['item']['lowPrice'])

        result.append(rtemp)
        
        
        
        #여기서부터는 데이터 저장
        
#파일이름 지정해주고 w모드로 열면 쓰기 가능       
f = open('갤럭시네이버 쇼핑검색.csv'.format(keyword) , 'w')


#해더값을 넣어준다.

#1은 첫 번째 페이지일 때를 뜻한다. 첫 번째 페이지의 데이터가 들어왔을 때만 헤더를 생성해 줘야 한다. 
#아니면 페이지를 만들 때마다 헤더가 생성돼서

#헤더
#데이터
#헤더
#데이터
#이런 식으로 들어가게 된다. 이러면 안 되기 때문에 첫 번째만 넣어줌
if pagenum == 1:
    f.write('상품이름' +',' + '최저가' + '\n')


for temp in result:    
        #이렇게하면 리스트로 받아오는데
        #리스트를 출력해보면
        #[제목,가격] 이렇게 받아온다.
        # 제목만 받아오려면 temp[0]
        # 가격만 받아오려면 temp[1]
        print(temp)
#         print(temp[0]) 제목출력
#         print(temp[1]) 가격출력
        #형식을 이렇게 넣어주면 
        #제목, 가격 엔터
        #제목, 가격 엔터
        #제목, 가격 엔터 이런식으로 들어간다.
        # str을 해준이유는 + 는 문자형에서만 쓸 수 있다. 제목은 문자만 들어있겠지만 
        #혹시 숫자가 들어가 있을수도 있기 때문에 문자형으로 변환해준다..
        # 가격은 숫자이기때문에 문자로 변환해준다.
        f.write(str(temp[0]) + ',' + str(temp[1]) + '\n')
       
f.close()

print('------------------------------------------------------------------')


# 네이버 블로그 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://m.blog.naver.com/woojehee00/222482005992'

page = requests.get(url)
html = page.text
html

In [ ]:
soup = BeautifulSoup(html,'lxml')
#링크로 들어가서 태그를 살펴보면 title태그가 떡하니 있음
titledata = soup.find('title')
#출력하면 제목이 나온다.
titledata.text

In [ ]:
titledata = soup.find('div',{'class':'se-section se-section-documentTitle se-l-default se-section-align-center'})
title = titledata.find('span').text

In [ ]:
#모든 내용이 다들어있는곳이 있음. 마우스로 잘 찾기
ddata = soup.find('div',{'class':'se-main-container'})
ddata

In [ ]:
#태그 계속 열어서 들어가보면 안에 se-module se-module-text가 있음
#그 안에 텍스트가 가득함
tdata = ddata.find_all('div',{'class':'se-module se-module-text'})
#두가지 방법이있음 둘중하나 이용
tdata = ddata.find_all('div',{'class':'se-component se-text se-l-default'})

result = ''

for temp in tdata:   
    #strip()은 문자열의 공백이 제거된다.괄호안에 지우고 싶은거 넣으면 지워진다.
    temp1 = temp.text.strip()    
    #re.sub로 영어 숫자 문자 빼고는 다 제거 한마디로 특수문자 제거
    temp1 = re.sub('[^A-Za-z0-9가-힣]',' ',temp1)
    
    #만약 temp1이 공백이 아니면 결과에 넣고 공백이면 다음 문자열을 가져온다.
    if temp1 != ' ':
        result += temp1
    result += ' '
    
result